In [2]:
!pip install -q chromadb
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers
!pip install -q langchain
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q bitsandbytes
!pip install -q trl
!pip install -q sentence_transformers
!pip install -q pypdf
!pip install -q unstructured
!pip install -q lark

In [2]:
!transformers-cli env


Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.39.2
- Platform: Windows-10-10.0.22621-SP0
- Python version: 3.11.8
- Huggingface_hub version: 0.22.1
- Safetensors version: 0.4.2
- Accelerate version: 0.28.0
- Accelerate config: 	not found
- PyTorch version (GPU?): 2.2.2+cu121 (True)
- Tensorflow version (GPU?): not installed (NA)
- Flax version (CPU?/GPU?/TPU?): not installed (NA)
- Jax version: not installed
- JaxLib version: not installed
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



In [1]:
import os
import torch
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
#################################################################
# Tokenizer
#################################################################

model_name='meta-llama/Meta-Llama-3-8B-Instruct'
#model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "bfloat16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Device partition
device_map = "auto"

In [14]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [15]:
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage = True,
    device_map = device_map
)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.99s/it]


In [30]:
#################################################################
# Base inference
#################################################################

prompt = """En que consiste el Artículo 7 de la ley 11/2022 del BOE de españa"""

# CUDA: Para programar directamente la GPU
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


En que consiste el Artículo 7 de la ley 11/2022 del BOE de españa?

Artículo 7 de la Ley 11/2022, publicada en el Boletín Oficial del Estado (BOE) de España, establece las condiciones para la aplicación de la exención de IVA en el suministro de energía eléctrica, gas natural y otros gases, así como el suministro de agua potable y de aguas residuales.

Según este artículo, para que una empresa pueda disfrutar de la exención de IVA en el suministro de energía eléctrica, gas natural y otros gases, debe cumplir con las siguientes condiciones:

1. Ser titular de una concesión, autorización o permiso expedido por las competentes autoridades para el suministro de energía eléctrica, gas natural o otros gases.
2. Ser titular de una licencia o permiso expedido por las competentes autoridades para el suministro de agua potable o aguas residuales.
3. No tener por objeto la venta al


## LangChain

In [2]:
import pypdf
from langchain.document_loaders import DirectoryLoader, PyPDFLoader, PDFMinerLoader
from pypdf import PdfReader
from langchain_community.document_loaders import UnstructuredXMLLoader

CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [3]:
def get_pdf_text(path):
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    
    return pages

def get_xml_text(path):
    loader = UnstructuredXMLLoader(
        path,
        mode = "elements", #Divide el report en cada uno de los campos del xml y crea un Document para cada uno
        strategy="fast",
        metadata_columns=["subtype", "type", "chief_complaint", "admit_diagnosis", "discharge_diagnosis", "year", "downlaod_time", "deid"],
    )
    docs = loader.load()
    return docs

In [4]:
from bs4 import BeautifulSoup
import os

def XMLLoader(folder_path):
    reportData_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                content = file.read()

            soup = BeautifulSoup(content, 'xml')

            reportData = {}
            for tag in soup.find_all():
                if tag.name != "report":
                    reportData[tag.name] = tag.text.strip()

            reportData["filename"] = filename
            reportData_list.append(reportData)

    return reportData_list

In [5]:
data =  XMLLoader("data")

In [6]:
from langchain.docstore.document import Document

documents = []
for doc in data:
    
    document = Document(page_content=doc["report_text"], metadata={"source": doc["filename"], 
                                                                   "subtype": doc["subtype"], 
                                                                   "type": doc["type"], 
                                                                   "chief_complaint": doc["chief_complaint"], 
                                                                   "admit_diagnosis": doc["admit_diagnosis"], 
                                                                   "discharge_diagnosis": doc["discharge_diagnosis"], 
                                                                   "year": doc["year"], 
                                                                   "downlaod_time": doc["downlaod_time"], 
                                                                   "deid": doc["deid"]})
    documents.append(document)

In [7]:
#documents[1].metadata["subtype"]="TEST"
#print(documents[7])

### Self retriever

In [8]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(documents, embeddings)

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
print(vectordb)

In [13]:
import pprint

doc_example = vectordb.get(ids=vectordb.get()['ids'][0])
for key in doc_example:

    print(f" ----  Start {key} --- \n" )
    pprint.pprint(doc_example[key])
    print(f" ----  End {key} --- \n" )

 ----  Start ids --- 

['044d8d7a-a672-4c6e-847d-a7bd6468a676']
 ----  End ids --- 

 ----  Start embeddings --- 

None
 ----  End embeddings --- 

 ----  Start metadatas --- 

[{'admit_diagnosis': '782.1',
  'chief_complaint': 'ALLERGIC REACTION',
  'deid': 'v.6.22.06.0',
  'discharge_diagnosis': '693.0,E931.0,709.8,781.1,V45.89,681.00,041.10,311,',
  'downlaod_time': '2008-02-06',
  'source': 'report25.xml',
  'subtype': '',
  'type': 'ER',
  'year': '2007'}]
 ----  End metadatas --- 

 ----  Start documents --- 

['[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.06.0]\n'
 '\n'
 '\n'
 ' \n'
 'EMERGENCY DEPARTMENT\n'
 'PATIENT RECORD\n'
 '**INSTITUTION\n'
 '**PLACE, PA  **ZIP-CODE\n'
 '_____________________________________________________\n'
 ' \n'
 '**NAME[AAA, BBB]\n'
 '**ID-NUM\n'
 'AERM \n'
 ' \n'
 ' \n'
 'DATE OF SERVICE               **DATE[Mar 12 06] \n'
 ' \n'
 'CHIEF COMPLAINT:  Rash.\n'
 ' \n'
 'HISTORY OF PRESENT ILLNESS:  The patient had an incision and drain

In [16]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

def load_model():
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.1,
        repetition_penalty=1.1,
        return_full_text=True,
        max_new_tokens=300,
        do_sample=True
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf

In [17]:
llm = load_model()

In [18]:
metadata_field_info = [
    AttributeInfo(
        name="checksum",
        description="Special identifier",
        type="string",
    ),
    AttributeInfo(
        name="subtype",
        description="Specific type of medical report",
        type="string",
    ),
    AttributeInfo(
        name="type",
        description="The type of medical report",
        type="string",
    ),
    AttributeInfo(
        name="chief_complaint",
        description="Main reason or purpose of the visit",
        type="string",
    ),
    AttributeInfo(
        name="admit_diagnosis",
        description="Primary diagnosis or reason for admission to the hospital",
        type="integer",
    ),
    AttributeInfo(
        name="discharge_diagnosis",
        description="Condition the patient is diagnosed with at the discharge from a hospital",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="Year of the report",
        type="integer",
    ),
    AttributeInfo(
        name="downlaod_time",
        description="Date of the dowload of the report",
        type="string",
    ),
    AttributeInfo(
        name="deid",
        description="De-identification data",
        type="string",
    ),
    AttributeInfo(
        name="source",
        description="Source report",
        type="string",
    )
]

document_content_desription = "Medical report for a hospital patient."

self_query_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore =vectordb,
    document_contents = document_content_desription,
    metadata_field_info = metadata_field_info,
    verbose = True,
    # structured_query_translator = ChromaTranslator()
)

In [24]:
self_query_retriever.invoke("What reports have a chief complaint of SEPSIS?")

OutputParserException: Parsing text
Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | 

In [19]:
from langchain_core.prompts import ChatPromptTemplate

TEMPLATE = """
[INST]
You are an expert doctor. Use the medical reports provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}

[/INST]
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [21]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": self_query_retriever })
output_parser = StrOutputParser()

self_retrieval_chain = setup_and_retrieval | rag_prompt | llm | output_parser

In [22]:
self_retrieval_chain.invoke( "What reports have a chief complaint of SEPSIS")

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\llama\modeling_llama.py:671: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


OutputParserException: Parsing text
Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | 

SelfQueryRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002E5A603DE90>, query_constructor=RunnableBinding(bound=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{{\n    "query": "teenager love",\n    "filter": "and(or(eq(\\"artist\\", \\"Taylor Swift\\"), eq(\\"artist\\", \\"Katy Perry\\"))